In [1]:
import re
import glob
import xlwings as xw
import math
from pathlib import Path
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as mclr
import plotly.express as px
import statsmodels.api as sm
import pylab as py
import os, cv2, glob, tempfile
import joblib

from scipy import stats
from scipy.stats import pearsonr

import sklearn
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.datasets import make_blobs

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import metrics
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
make_scorer = sklearn.metrics.make_scorer
f1 = make_scorer(f1_score, pos_label=1, average="binary")
from sklearn.metrics import classification_report
from sklearn.inspection import permutation_importance
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import RepeatedStratifiedKFold

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras import utils as np_utils
from keras import models
import keras_tuner as kt
from scipy import signal

import config as config
print(config.__path__)
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras import backend as K

import argparse
# from sklearn.utils import class_weight
print(tf.__version__)
print(keras.__version__)

import shutil
import xlwings as xw

%matplotlib inline
mpl.use("Agg")

['C:\\Users\\user\\anaconda3\\lib\\site-packages\\config']
2.12.0-dev20221107
2.12.0


In [2]:
## Build XGBoost Model
from copy import deepcopy
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,train_test_split
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score,confusion_matrix,roc_curve
import xgboost as xgb
import time
import warnings

# 1. Data Preparation

In [3]:
trainDEFSDf = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_train_dbMSIn5ppm4nonInDI_norm.csv")
diDEFSDf = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_nonInDI_dbMSIn5ppm4nonInDI_norm.csv")

In [4]:
trainDEFSDf

,pixel_id,311.1684,269.2486,215.0328,295.2278,883.533,309.1725,738.5057,435.2965,280.2361,241.2173,311.2228,339.1996,353.2004,325.1842,250.1449,514.2846,265.1478,type
0,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,0.318763,0.150470,-0.119121,-0.067072,-0.071854,0.384412,-0.143360,0.118134,-0.082548,0.226547,-0.006497,0.230776,0.249742,0.293674,0.126164,-0.040567,0.322933,0
1,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,0.440853,0.394966,-0.038289,-0.067072,0.017921,0.479880,-0.160340,0.058233,-0.102401,0.292198,-0.006497,0.312503,0.313633,0.399218,0.300476,-0.040567,0.392722,0
2,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,0.297858,0.166363,-0.083697,-0.067072,-0.069362,0.317106,-0.150742,-0.029448,-0.088097,0.210543,-0.006497,0.229695,0.257632,0.267805,0.208799,-0.040567,0.315268,0
3,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,0.276322,0.115277,-0.042454,-0.067072,-0.054074,0.299065,-0.143111,-0.018081,-0.106709,0.272757,-0.006497,0.221038,0.208418,0.247311,0.187170,-0.040567,0.303836,0
4,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,0.172632,0.200346,-0.131919,-0.067072,-0.052373,0.219220,-0.146138,0.140049,-0.064402,0.166744,-0.006497,0.138928,0.194134,0.151399,0.094790,-0.040567,0.213324,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90955,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.035493,-0.272607,0.004148,-0.067072,-0.142422,0.062967,-0.200362,0.242729,-0.173635,-0.248759,-0.006497,0.042836,0.066402,0.035222,0.038175,0.212752,0.036053,1
90956,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.021826,-0.272607,-0.091252,-0.067072,-0.062065,0.074304,-0.191751,0.300467,-0.195074,0.152673,-0.006497,0.050344,0.070289,0.037014,0.210629,-0.040567,0.018591,1
90957,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.030907,0.332829,-0.093406,-0.067072,-0.090889,0.044570,-0.114260,0.195776,-0.156608,-0.248759,-0.006497,0.046817,0.103054,0.037404,0.143302,-0.040567,-0.052554,1
90958,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.045524,-0.272607,-0.122689,-0.067072,-0.163137,0.059988,-0.138465,0.149654,-0.163002,-0.248759,-0.006497,0.033911,-0.067550,0.036384,0.189038,-0.040567,0.048043,1


In [5]:
can = []
for i in list(range(0,9)) + list(range(15,17)) + [len(list(trainDEFSDf.columns))-1]:
    can.append(list(trainDEFSDf.columns)[i])

dican = []
for i in list(range(0,9)) + list(range(15,17)) + [len(list(diDEFSDf.columns))-1]:
    dican.append(list(diDEFSDf.columns)[i])

In [6]:
can

['pixel_id',
 '311.1684',
 '269.2486',
 '215.0328',
 '295.2278',
 '883.533',
 '309.1725',
 '738.5057',
 '435.2965',
 '250.1449',
 '514.2846',
 'type']

In [7]:
dican

['pixel_id',
 '311.169',
 '269.2486',
 '215.032',
 '295.2279',
 '883.5375',
 '309.1728',
 '738.5084',
 '435.2966',
 '250.145',
 '514.2844',
 'type']

In [8]:
## input training set ## 90960 x 20 df
trainDEFSDf = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_train_dbMSIn5ppm4nonInDI_norm.csv")
#trainDEFSDf[trainDEFSDf.type .== 1, :]
## calculate weight ## 0: 47449, 1: 43511
trainDEFSDf = trainDEFSDf[can]
Yy_train = trainDEFSDf["type"]  # 0.9585; 1.0453
sampleW = []
for w in Yy_train:
    if w == 0:
        sampleW.append(0.9585)
    elif w == 1:
        sampleW.append(1.0453) 


In [9]:
## input ext val set ## 6075 x 20 df
extDEFSDf = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_ext_dbMSIn5ppm4nonInDI_norm.csv")
#extDEFSDf[extDEFSDf.type .== 1, :]
## calculate weight ## 0: 2943, 1: 3132
extDEFSDf = extDEFSDf[can]
Yy_ext = extDEFSDf["type"]  # 1.0321; 0.9698
sampleExtW = []
for w in Yy_ext:
    if w == 0:
        sampleExtW.append(1.0321)
    elif w == 1:
        sampleExtW.append(0.9698) 


In [10]:
## input ingested set ## 97035 x 20 df
ingestedDEFSDf = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_ingested_dbMSIn5ppm4nonInDI_norm.csv")
#ingestedDEFSDf[extDEFSDf.type .== 1, :]
## calculate weight ## 0: 50392, 1: 46643
ingestedDEFSDf = ingestedDEFSDf[can]
Yy_ingested = ingestedDEFSDf["type"]  # 0.9628; 1.0402
sampleIngestedW = []
for w in Yy_ingested:
    if w == 0:
        sampleIngestedW.append(0.9628)
    elif w == 1:
        sampleIngestedW.append(1.0402)


In [11]:
## input FNA set ## 88701 x 20 df
fnaDEFSDf = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_FNA_dbMSIn5ppm4nonInDI_norm.csv")
#fnaDEFSDf[fnaDEFSDf.type .== 1, :]
## calculate weight ##  0: 44540, 1: 44161
fnaDEFSDf = fnaDEFSDf[can]
Yy_FNA = fnaDEFSDf["type"]  # 0.9957; 1.0043
sampleFNAW = []
for w in Yy_FNA:
    if w == 0:
        sampleFNAW.append(0.9957)
    elif w == 1:
        sampleFNAW.append(1.0043)


In [12]:
## input DirectIn set ## 88701 x 20 df
diDEFSDf = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_nonInDI_dbMSIn5ppm4nonInDI_norm.csv")
#diDEFSDf[diDEFSDf.type .== 1, :]
## calculate weight ## 0: 3027, 1: 3030
diDEFSDf = diDEFSDf[dican]
diDEFSDf = diDEFSDf.rename(columns={"311.169": "311.1684", "215.032":"215.0328", "295.2279":"295.2278", "883.5375":"883.533", 
                                    "309.1728":"309.1725", "738.5084":"738.5057", "435.2966":"435.2965", "280.2364":"280.2361", "241.217":"241.2173", 
                                    "311.2224":"311.2228", "339.1999":"339.1996", "353.1996":"353.2004", "325.1844":"325.1842", "250.145":"250.1449", 
                                    "514.2844":"514.2846", "265.1479":"265.1478"})
Yy_DI = diDEFSDf["type"]  # 1.0005; 0.9995
sampleDiW = []
for w in Yy_DI:
    if w == 0:
        sampleDiW.append(1.0005)
    elif w == 1:
        sampleDiW.append(0.9995)


In [13]:
diDEFSDf

,pixel_id,311.1684,269.2486,215.0328,295.2278,883.533,309.1725,738.5057,435.2965,250.1449,514.2846,type
0,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.005233,0.142288,0.041727,0.003879,0.078180,0.211155,0.061859,0.125898,0.467350,0.036498,0
1,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.089591,0.156412,0.177387,0.001265,0.167206,0.317443,0.128000,0.145558,0.583306,0.099618,0
2,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.032985,0.134206,0.036969,-0.069655,0.122163,0.212954,0.028988,0.076856,0.465193,0.056825,0
3,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.081001,0.235152,0.064473,0.057514,0.129276,0.303459,0.108118,0.193038,0.550698,0.090339,0
4,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.132548,0.236938,0.155154,0.080273,0.239452,0.394214,0.236248,0.204588,0.645501,0.147333,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6052,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.002046,-0.209455,0.129981,-0.039157,-0.042339,-0.056631,0.023269,0.147197,-0.089365,0.163267,1
6053,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.313188,-0.217983,-0.306250,-0.299289,-0.272830,-0.303023,-0.280528,-0.277815,-0.025231,-0.251944,1
6054,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.158181,-0.037213,0.001807,-0.048526,-0.044929,-0.153125,-0.084889,-0.039383,-0.089365,0.091528,1
6055,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.130111,-0.100314,0.068308,-0.107947,-0.006630,-0.127231,-0.068977,-0.070617,-0.010167,-0.018040,1


In [14]:
## define functions for performace evaluation ##

# Average score
def avgScore(arrAcc, cv):
    sumAcc = 0
    for acc in arrAcc:
        sumAcc += acc
    return sumAcc / cv


In [15]:
trainDEFSDf[trainDEFSDf.columns[1:-1]]

,311.1684,269.2486,215.0328,295.2278,883.533,309.1725,738.5057,435.2965,250.1449,514.2846
0,0.318763,0.150470,-0.119121,-0.067072,-0.071854,0.384412,-0.143360,0.118134,0.126164,-0.040567
1,0.440853,0.394966,-0.038289,-0.067072,0.017921,0.479880,-0.160340,0.058233,0.300476,-0.040567
2,0.297858,0.166363,-0.083697,-0.067072,-0.069362,0.317106,-0.150742,-0.029448,0.208799,-0.040567
3,0.276322,0.115277,-0.042454,-0.067072,-0.054074,0.299065,-0.143111,-0.018081,0.187170,-0.040567
4,0.172632,0.200346,-0.131919,-0.067072,-0.052373,0.219220,-0.146138,0.140049,0.094790,-0.040567
...,...,...,...,...,...,...,...,...,...,...
90955,0.035493,-0.272607,0.004148,-0.067072,-0.142422,0.062967,-0.200362,0.242729,0.038175,0.212752
90956,0.021826,-0.272607,-0.091252,-0.067072,-0.062065,0.074304,-0.191751,0.300467,0.210629,-0.040567
90957,0.030907,0.332829,-0.093406,-0.067072,-0.090889,0.044570,-0.114260,0.195776,0.143302,-0.040567
90958,0.045524,-0.272607,-0.122689,-0.067072,-0.163137,0.059988,-0.138465,0.149654,0.189038,-0.040567


In [16]:
import warnings
warnings.filterwarnings('ignore')

# 2. Unsupervised Learning

## 2.1. Dataset Preparation

In [17]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

In [18]:
df_ROI_for_ML_Opti = pd.concat([trainDEFSDf, extDEFSDf, fnaDEFSDf, diDEFSDf]).set_index('pixel_id')

In [19]:
df_ROI_for_ML_Opti_ingested = ingestedDEFSDf.set_index('pixel_id')

In [20]:
df_ROI_for_ML_Opti_train = trainDEFSDf.set_index('pixel_id')

In [21]:
df_ROI_for_ML_Opti_ext = extDEFSDf.set_index('pixel_id')

In [22]:
df_ROI_for_ML_Opti_FNA = fnaDEFSDf.set_index('pixel_id')

In [23]:
df_ROI_for_ML_Opti_di = diDEFSDf.set_index('pixel_id')

In [24]:
df_ROI_for_ML_Opti_di.describe()

,311.1684,269.2486,215.0328,295.2278,883.533,309.1725,738.5057,435.2965,250.1449,514.2846,type
count,6.057000e+03,6.057000e+03,6.057000e+03,6.057000e+03,6.057000e+03,6.057000e+03,6.057000e+03,6.057000e+03,6.057000e+03,6.057000e+03,6057.000000
mean,2.786097e-18,6.620646e-17,2.832104e-16,-1.438873e-18,-4.755611e-16,9.102472e-17,1.235597e-16,-4.213514e-16,-4.598716e-15,-7.857894e-17,0.500248
std,1.259709e-01,1.439456e-01,1.644526e-01,1.354221e-01,1.816907e-01,1.711848e-01,1.276984e-01,2.074127e-01,1.914162e-01,1.330110e-01,0.500041
min,-3.936976e-01,-4.211962e-01,-3.062504e-01,-4.545066e-01,-2.728296e-01,-3.622890e-01,-2.805282e-01,-2.778148e-01,-8.936520e-02,-3.327803e-01,0.000000
25%,-7.672668e-02,-6.810890e-02,-9.448816e-02,-6.188994e-02,-1.079244e-01,-8.982515e-02,-5.994488e-02,-2.778148e-01,-8.936520e-02,-6.633725e-02,0.000000
50%,9.552356e-03,-6.986512e-03,1.676521e-02,1.774746e-02,3.107505e-02,-3.331810e-02,1.461270e-02,6.133792e-03,-8.936520e-02,-1.853303e-02,1.000000
75%,9.000432e-02,5.958404e-02,1.127828e-01,7.451773e-02,1.306204e-01,3.533862e-02,6.957675e-02,1.666052e-01,-1.214997e-03,8.938771e-02,1.000000
max,6.063024e-01,5.788038e-01,6.937496e-01,5.454934e-01,7.271704e-01,6.377110e-01,7.194718e-01,7.221852e-01,9.106348e-01,6.672197e-01,1.000000


## 2.2. PCA Analysis

In [37]:
## PCA Analysis

def pca_visual(df=df_ROI_for_ML_Opti_di):

    pca = PCA(n_components=3)
    raMSI_ML_mz_df = df.drop("type", axis = 1)
    pca.fit(raMSI_ML_mz_df)
    result = pd.DataFrame(pca.transform(raMSI_ML_mz_df), columns=['PCA%i' % i for i in range(3)], index=df_ROI_for_ML_Opti_di.index)
    
    plt.subplots(dpi = 300)
    plt.scatter(result['PCA1'], result['PCA2'], c = df['type'], s=1)
    plt.xlabel('PC1', size=20)
    plt.ylabel('PC2', size=20)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/dbMSIn_5ppm/PCA_di.tif", bbox_inches = 'tight')

    components = pca.fit_transform(raMSI_ML_mz_df)
    total_var = pca.explained_variance_ratio_.sum() * 100
    fig = px.scatter_3d(
        components, x=0, y=1, z=2, color=df['type'],
        title = f'Total Explained Variance: {total_var:.2f}%',
        labels = {'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
    )
    fig.show()

In [ ]:
pca_visual()

# 3. Supervised Learning

## 3.1. Preparation of training and test dataset

In [39]:
def split_dataset(df=df_ROI_for_ML_Opti_ingested):
    raMSI_ML_NoType = df.drop("type", axis = 1)
    raMSI_ML_Type=df[['type']]
    X_train,X_test,y_train,y_test=train_test_split(raMSI_ML_NoType,raMSI_ML_Type,test_size=0.2,random_state=42)
    return X_train,X_test,y_train,y_test

X_train,X_test,y_train,y_test=split_dataset(df=df_ROI_for_ML_Opti)

In [40]:
X_ingested = df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[:-1]]

In [41]:
y_ingested = df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[-1]]

In [42]:
X_train = df_ROI_for_ML_Opti_train[df_ROI_for_ML_Opti_train.columns[:-1]]

In [43]:
y_train = df_ROI_for_ML_Opti_train[df_ROI_for_ML_Opti_train.columns[-1]]

In [44]:
X_val = df_ROI_for_ML_Opti_ext[df_ROI_for_ML_Opti_ext.columns[:-1]]

In [45]:
y_val = df_ROI_for_ML_Opti_ext[df_ROI_for_ML_Opti_ext.columns[-1]]

In [46]:
X_FNA = df_ROI_for_ML_Opti_FNA[df_ROI_for_ML_Opti_FNA.columns[:-1]]

In [47]:
y_FNA = df_ROI_for_ML_Opti_FNA[df_ROI_for_ML_Opti_FNA.columns[-1]]

In [48]:
X_di = df_ROI_for_ML_Opti_di[df_ROI_for_ML_Opti_di.columns[:-1]]

In [49]:
y_di = df_ROI_for_ML_Opti_di[df_ROI_for_ML_Opti_di.columns[-1]]

## 3.2 Machine Learning Modeling: Model 1+2 (Train 10:10, Val 4:4, Test 3:3)

In [50]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

### 3.2.1. Logistic Regression

In [51]:
## Build Logistic Regression Model
from sklearn.linear_model import LogisticRegression

def LogisticRegression_func(XTrain=X_ingested, XVal=X_val, XTest=X_FNA, yTrain=y_ingested, yVal=y_val, yTest=y_FNA):
    model_lr = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})  # max_iter=100
    model_lr.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_lr.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_lr.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_lr.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_lr.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_lr.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_lr.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_lr, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_lr.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_lr.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_lr.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_lr.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    return x, y

In [52]:
coeff_LogReg, TOPfeatures_LogReg = LogisticRegression_func()

Training Set F1: 0.8406548865609851
Training Set MCC: 0.6813608572119455
Training Set Accuracy: 0.8406882145998241
              precision    recall  f1-score   support

           0       0.85      0.84      0.85     47449
           1       0.83      0.84      0.83     43511

    accuracy                           0.84     90960
   macro avg       0.84      0.84      0.84     90960
weighted avg       0.84      0.84      0.84     90960

[[39898  7551]
 [ 6940 36571]]
Ext Val Set F1: 0.8152216111756206
Ext Val Set MCC: 0.6040424643656963
Ext Val Set Accuracy: 0.7973662551440329
              precision    recall  f1-score   support

           0       0.88      0.68      0.76      2943
           1       0.75      0.91      0.82      3132

    accuracy                           0.80      6075
   macro avg       0.81      0.79      0.79      6075
weighted avg       0.81      0.80      0.79      6075

[[1994  949]
 [ 282 2850]]
5F-CV: 0.6279461455926231
FNA Set F1: 0.746655739208128
FNA S

### 3.2.2. SVM

In [53]:
## Build LinearSVC Model
from sklearn.svm import LinearSVC

def LinearSVC_func(XTrain=X_ingested, XVal=X_val, XTest=X_FNA, yTrain=y_ingested, yVal=y_val, yTest=y_FNA):
    model_svm = LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})
    model_svm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_svm.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_svm.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_svm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_svm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_svm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_svm.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_svm.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    return x, y

In [54]:
coeff_SVM, TOPfeatures_SVM = LinearSVC_func()

Training Set F1: 0.8847064904390333
Training Set MCC: 0.7647722222645835
Training Set Accuracy: 0.880914687774846
              precision    recall  f1-score   support

           0       0.91      0.86      0.88     47449
           1       0.85      0.91      0.88     43511

    accuracy                           0.88     90960
   macro avg       0.88      0.88      0.88     90960
weighted avg       0.88      0.88      0.88     90960

[[40714  6735]
 [ 4097 39414]]
Ext Val Set F1: 0.8287434038066259
Ext Val Set MCC: 0.6468450640451637
Ext Val Set Accuracy: 0.8237037037037037
              precision    recall  f1-score   support

           0       0.84      0.79      0.81      2943
           1       0.81      0.86      0.83      3132

    accuracy                           0.82      6075
   macro avg       0.83      0.82      0.82      6075
weighted avg       0.82      0.82      0.82      6075

[[2315  628]
 [ 443 2689]]
5F-CV: 0.9159689638386391
FNA Set F1: 0.6505794800746774
FNA S

### 3.2.3. GBM

In [55]:
## Build GBM Model
from sklearn.ensemble import GradientBoostingClassifier

def GBM_func(XTrain=X_ingested, XVal=X_val, XTest=X_FNA, yTrain=y_ingested, yVal=y_val, yTest=y_FNA):
    model_gbm = GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)
    model_gbm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_gbm.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_gbm.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_gbm.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_gbm.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_gbm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_gbm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_gbm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_gbm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_gbm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_gbm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_gbm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_gbm.feature_importances_, index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/XGB_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()

    # Visualize the boxplot of the important features between the two groups
    #fig = px.box(data_frame = df_ROI_for_ML_Opti, x = feature_imp[:10].index,color= 'type')
    #fig.show()
    
    return x, y

In [56]:
coeff_GBM, TOPfeatures_GBM = GBM_func()

Training Set F1: 0.44491498896790316
Training Set MCC: 0.3058405860365625
Training Set Accuracy: 0.6326517150395778
              precision    recall  f1-score   support

           0       0.59      0.93      0.73     47449
           1       0.81      0.31      0.44     43511

    accuracy                           0.63     90960
   macro avg       0.70      0.62      0.58     90960
weighted avg       0.70      0.63      0.59     90960

[[44261  3188]
 [30226 13285]]
Ext Val Set F1: 0.36669374228601226
Ext Val Set MCC: -0.08672126955572901
Ext Val Set Accuracy: 0.4539917695473251
              precision    recall  f1-score   support

           0       0.45      0.60      0.52      2943
           1       0.46      0.31      0.37      3132

    accuracy                           0.45      6075
   macro avg       0.45      0.46      0.44      6075
weighted avg       0.45      0.45      0.44      6075

[[1776 1167]
 [2150  982]]
5F-CV: 0.6032422440438727
FNA Set F1: 0.42159802760103754

### 3.2.4. Stacking

In [57]:
class StackingRegressor():
    
    def __init__(self, learners):
        self.level_sizes = []
        self.learners = []
        
        for learning_level in learners:
            self.level_sizes.append(len(learning_level))
            level_learners = []
            
            for learner in learning_level:
                level_learners.append(deepcopy(learner))
            self.learners.append(level_learners)
            
            
    def fit(self, x, y):
        
        meta_data = [x,x,x]
        meta_targets = [y,y,y]
        
        for i in range(len(self.learners)):
            level_size = self.level_sizes[i]
            
            data_z = np.zeros((level_size, len(x)))
            target_z = np.zeros(len(x))
            
            train_x = meta_data[i]
            train_y = meta_targets[i]
            
            # Define number of folds
            num_folds = 5
            
            # Create the k-fold cross-validation object
            KF = KFold(n_splits=num_folds)
            m = 0

            # Loop over each fold of the cross-validation
            for train_indices, test_indices in KF.split(x):
                
                for j in range(len(self.learners[i])):
                    train_x = pd.DataFrame(train_x)
                    train_y = pd.DataFrame(train_y)

                    learner = self.learners[i][j]
                    learner.fit(train_x.iloc[train_indices], train_y.iloc[train_indices])
                    p = learner.predict(train_x.iloc[test_indices])
                    data_z[j][m: m+len(test_indices) ] = p


                train_y = pd.DataFrame(train_y)
                train_y_array = (np.array(train_y)).reshape(-1,)
                zty_ind = (np.array(train_y.iloc[test_indices])).reshape(-1,)
                target_z[m: m+len(test_indices)] = train_y_array[zty_ind]
                m += len(test_indices)
                

            data_z = data_z.transpose()
            meta_data.append(data_z)
            meta_targets.append(target_z)
            
            
            for learner in self.learners[i]:
                train_x = pd.DataFrame(train_x)
                train_y = pd.DataFrame(train_y)
                learner.fit(train_x, train_y)
                
                
    def predict(self, x):
        
        meta_data = [x,x,x]
        for i in range(len(self.learners)):
            
            level_size = self.level_sizes[i]
            
            data_z = np.zeros((level_size, len(x)))
            
            test_x = meta_data[i]
            
            for j in range(len(self.learners[i])):
                
                learner = self.learners[i][j]
                predictions = learner.predict(test_x)
                data_z[j] = predictions
                
            
            data_z = data_z.transpose()
            meta_data.append(data_z)
            
        return meta_data


In [58]:
## Build stacking Model

def Stacking_func(XTrain=X_ingested, XVal=X_val, XTest=X_FNA, XTest2=X_di, yTrain=y_ingested, yVal=y_val, yTest=y_FNA, yTest2=y_di):

    base_learners = []

    # Define models to use in stacking ensemble
    m1 = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})
    base_learners.append(m1)

    m2 = LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})
    base_learners.append(m2)

    m3 = GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)
    base_learners.append(m3)

    # Define the meta-classifier to use for stacking
    #meta_learner = LinearDiscriminantAnalysis()  # 0.90,0.80; 0.75,0.51; 0.65,0.27; 0.64,0.26
    #meta_learner = QuadraticDiscriminantAnalysis()  # 0.93,0.86; 0.67,0.39; 0.67,0.15; 0.68,0.20
    
    #meta_learner = LogisticRegression()  # 0.90,0.79; 0.82,0.65; 0.64,0.24; 0.64;0.27
    #meta_learner = LogisticRegression(C=1, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=1: 0.90,0.79; 0.82,0.65; 0.65,0.25; 0.65;0.27
    #meta_learner = LogisticRegression(C=0.1, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.1: 0.90,0.79; 0.82,0.65; 0.65,0.25; 0.65;0.28
    #meta_learner = LogisticRegression(C=0.01, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.01: 0.89,0.78; 0.81,0.63; 0.63,0.22; 0.65;0.29
    #meta_learner = LogisticRegression(C=0.0075, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.0075: 0.89,0.77; 0.81,0.63; 0.62,0.20; 0.65;0.29
    #meta_learner = LogisticRegression(C=0.005, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.005: 0.88,0.76; 0.82,0.64; 0.60,0.17; 0.66;0.31
    #meta_learner = LogisticRegression(C=0.0025, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.0025: 0.86,0.72; 0.83,0.66; 0.53,0.02; 0.64;0.31
    #meta_learner = LogisticRegression(C=0.0015, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.0015: 0.85,0.70; 0.80,0.60; 0.55,0.09; 0.64;0.30
    #meta_learner = LogisticRegression(C=0.0013, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.0013: 0.85,0.70; 0.78,0.56; 0.58,0.16; 0.63;0.29
    #meta_learner = LogisticRegression(C=0.0012, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.0012: 0.85,0.69; 0.77,0.52; 0.60,0.22; 0.63;0.27
    meta_learner = LogisticRegression(C=0.0011, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.0011: 0.85,0.69; 0.77,0.51; 0.64,0.30; 0.62;0.25
    #meta_learner = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.001: 0.84,0.68; 0.76,0.49; 0.68,0.38; 0.61;0.22
    #meta_learner = LogisticRegression(C=0.0005, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.0005: 0.83,0.66; 0.74,0.45; 0.65,0.37; 0.60;0.18
    
    #meta_learner = LinearSVC()  # 0.90,0.80; 0.82,0.65; 0.64,0.24; 0.65,0.27
    #meta_learner = KNeighborsClassifier()  # 0.98,0.96; 0.65,0.30; 0.68,0.21; 0.52,-0.05
    #meta_learner = DecisionTreeClassifier()  # 0.94,0.87; 0.54,0.15; 0.66,0.11; 0.46,-0.11
    #meta_learner = RandomForestClassifier()  # 0.98,0.95; 0.54,0.20; 0.66,0.11; 0.49;-0.01
    #meta_learner = GradientBoostingClassifier()  # 0.97,0.94; 0.56,0.27; 0.66,0.13; 0.48,-0.09
    #meta_learner = AdaBoostClassifier()  # 0.96,0.93; 0.69,0.42; 0.62,0.10; 0.54,0.11
    #meta_learner = xgb.XGBClassifier()  # 0.98,0.95; 0.59,0.30; 0.66,0.10; 0.44,-0.12

    model_sc = StackingRegressor([[m1, m2, m3], [meta_learner]])
    model_sc.fit(XTrain, yTrain)
    
    #=======================================
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_sc.predict(X_train)[-1]
    print("Training Set F1:", f1_score(y_train, model_sc.predict(X_train)[-1], sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_sc.predict(X_train)[-1], sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_sc.predict(XVal)[-1]
    print("Ext Val Set F1:", f1_score(yVal, model_sc.predict(XVal)[-1], sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_sc.predict(XVal)[-1], sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    #f1_5_train = cross_val_score(model_sc, XTrain, yTrain, cv = 5, scoring="f1")
    #print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_sc.predict(XTest)[-1]
    print("FNA Set F1:", f1_score(yTest, model_sc.predict(XTest)[-1], sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_sc.predict(XTest)[-1], sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_sc.predict(XTest)[-1]))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Test2 set
    predictions_di = model_sc.predict(XTest2)[-1]
    print("DirectIn Set F1:", f1_score(yTest2, model_sc.predict(XTest2)[-1], sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, model_sc.predict(XTest2)[-1], sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, model_sc.predict(XTest2)[-1]))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    

In [59]:
Stacking_func()

Training Set F1: 0.8423672428013455
Training Set MCC: 0.6846344917985415
Training Set Accuracy: 0.8423043095866315
              precision    recall  f1-score   support

           0       0.85      0.84      0.85     47449
           1       0.83      0.84      0.84     43511

    accuracy                           0.84     90960
   macro avg       0.84      0.84      0.84     90960
weighted avg       0.84      0.84      0.84     90960

[[39953  7496]
 [ 6848 36663]]
Ext Val Set F1: 0.832634364655037
Ext Val Set MCC: 0.644872636681615
Ext Val Set Accuracy: 0.8190946502057613
              precision    recall  f1-score   support

           0       0.89      0.72      0.79      2943
           1       0.77      0.92      0.84      3132

    accuracy                           0.82      6075
   macro avg       0.83      0.82      0.82      6075
weighted avg       0.83      0.82      0.82      6075

[[2109  834]
 [ 265 2867]]
FNA Set F1: 0.7108418398555316
FNA Set MCC: 0.4392628452832895


### 3.2.5. Voting

In [60]:
# get a voting ensemble of models
def get_voting():
    models = list()
    # Define the base models
    models.append(("m1", LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})))
    models.append(("m2", LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})))
    models.append(("m3", GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)))

    # define the voting ensemble
    ensemble = VotingClassifier(estimators=models, voting='hard')
    return ensemble

# get a list of models to evaluate
def get_models():
    models = dict()
    models['m1'] = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})
    models['m2'] = LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})
    models['m3'] = GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)
    models['hard_voting'] = get_voting()
    return models


# evaluate a give model using cross-validation
def evaluate_model(model, XX, yy):
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)
    scores = cross_val_score(model, XX, yy, scoring='f1', cv=cv, n_jobs=-1, error_score='raise')
    return scores


In [61]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

In [63]:
# get the models to evaluate
models = get_models()

# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, XX=X_ingested, yy=y_ingested)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, np.mean(scores), np.std(scores)))
    
# plot model performance for comparison
plt.subplots(dpi = 600)
plt.boxplot(results, labels=names, showmeans=True)
plt.ylabel('F1 score')
plt.xlabel('Predictive Models')
plt.savefig(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/dbMSIn_5ppm/voting.jpg", bbox_inches = 'tight')

>m1 0.811 (0.003)
>m2 0.933 (0.009)
>m3 0.671 (0.144)
>hard_voting 0.902 (0.022)


In [64]:
def Voting_func(XTrain=X_ingested, XVal=X_val, XTest=X_FNA, XTest2=X_di, yTrain=y_ingested, yVal=y_val, yTest=y_FNA, yTest2=y_di):

    models = get_models()
    
    ensemble = models["hard_voting"]
    ensemble.fit(XTrain, yTrain)
    #=======================================

    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = ensemble.predict(X_train)
    print("Training Set F1:", f1_score(y_train, ensemble.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, ensemble.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = ensemble.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, ensemble.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, ensemble.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()

    f1_5_train = cross_val_score(ensemble, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))

    # Test set
    predictions_FNA = ensemble.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, ensemble.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, ensemble.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, ensemble.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()

    # Test2 set
    predictions_di = ensemble.predict(XTest2)
    print("DirectIn Set F1:", f1_score(yTest2, ensemble.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, ensemble.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, ensemble.predict(XTest2)))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()

In [65]:
Voting_func()

Training Set F1: 0.8833318637005201
Training Set MCC: 0.7616614513328519
Training Set Accuracy: 0.8792106420404573
              precision    recall  f1-score   support

           0       0.91      0.85      0.88     47449
           1       0.85      0.91      0.88     43511

    accuracy                           0.88     90960
   macro avg       0.88      0.88      0.88     90960
weighted avg       0.88      0.88      0.88     90960

[[40549  6900]
 [ 4087 39424]]
Ext Val Set F1: 0.864749296675819
Ext Val Set MCC: 0.7228694161043941
Ext Val Set Accuracy: 0.8617283950617284
              precision    recall  f1-score   support

           0       0.88      0.83      0.85      2943
           1       0.85      0.89      0.87      3132

    accuracy                           0.86      6075
   macro avg       0.86      0.86      0.86      6075
weighted avg       0.86      0.86      0.86      6075

[[2448  495]
 [ 345 2787]]
5F-CV: 0.8109135421047764
FNA Set F1: 0.7203888994797493
FNA S

In [66]:
import joblib as jl

In [67]:
models = get_models()
    
ensemble = models["hard_voting"]
ensemble.fit(X_ingested, y_ingested)

VotingClassifier(estimators=[('m1',
                              LogisticRegression(C=0.001,
                                                 class_weight={0: 0.9628,
                                                               1: 1.0402},
                                                 max_iter=5000, penalty='l1',
                                                 random_state=42,
                                                 solver='liblinear')),
                             ('m2',
                              LinearSVC(C=461,
                                        class_weight={0: 0.9628, 1: 1.0402},
                                        random_state=42)),
                             ('m3',
                              GradientBoostingClassifier(learning_rate=4,
                                                         max_depth=7,
                                                         min_samples_leaf=4,
                                                         min_sampl

In [68]:
modelSavePath = r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\dbMSIn_5ppm\model3to1Voting_dbMSIn5ppm.joblib"
jl.dump(ensemble, modelSavePath, compress = 5)

['H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\afterModelSelection\\dbMSIn_5ppm\\model3to1Voting_dbMSIn5ppm.joblib']

## 3.3 Machine Learning Modeling: Model 1 (Train 6:6, Val 4:4, Test 3:3)

### 3.3.1. Logistic Regression

In [69]:
## Build Logistic Regression Model
from sklearn.linear_model import LogisticRegression

def LogisticRegression_func(XTrain=X_train, XVal=X_val, XTest=X_FNA, yTrain=y_train, yVal=y_val, yTest=y_FNA):
    model_lr = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})  # max_iter=100
    model_lr.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_lr.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_lr.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_lr.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_lr.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_lr.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_lr.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_lr, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_lr.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_lr.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_lr.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_lr.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    return x, y

In [70]:
coeff_LogReg, TOPfeatures_LogReg = LogisticRegression_func()

Training Set F1: 0.8607786920989818
Training Set MCC: 0.7177102858667281
Training Set Accuracy: 0.8580474934036939
              precision    recall  f1-score   support

           0       0.88      0.84      0.86     47449
           1       0.84      0.87      0.85     43511

    accuracy                           0.86     90960
   macro avg       0.86      0.86      0.86     90960
weighted avg       0.86      0.86      0.86     90960

[[40035  7414]
 [ 5498 38013]]
Ext Val Set F1: 0.7455915678272023
Ext Val Set MCC: 0.45450778489828036
Ext Val Set Accuracy: 0.7265843621399177
              precision    recall  f1-score   support

           0       0.76      0.64      0.69      2943
           1       0.70      0.81      0.75      3132

    accuracy                           0.73      6075
   macro avg       0.73      0.72      0.72      6075
weighted avg       0.73      0.73      0.72      6075

[[1880 1063]
 [ 598 2534]]
5F-CV: 0.7458898574536084
FNA Set F1: 0.5255329612418614
FNA

### 3.3.2. SVM

In [71]:
## Build LinearSVC Model
from sklearn.svm import LinearSVC

def LinearSVC_func(XTrain=X_train, XVal=X_val, XTest=X_FNA, yTrain=y_train, yVal=y_val, yTest=y_FNA):
    model_svm = LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})
    model_svm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_svm.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_svm.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_svm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_svm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_svm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_svm.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_svm.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    return x, y

In [72]:
coeff_SVM, TOPfeatures_SVM = LinearSVC_func()

Training Set F1: 0.9247348950913382
Training Set MCC: 0.8467684201619845
Training Set Accuracy: 0.9171943711521547
              precision    recall  f1-score   support

           0       0.98      0.86      0.92     47449
           1       0.86      0.98      0.92     43511

    accuracy                           0.92     90960
   macro avg       0.92      0.92      0.92     90960
weighted avg       0.93      0.92      0.92     90960

[[40634  6815]
 [  717 42794]]
Ext Val Set F1: 0.6054244661836908
Ext Val Set MCC: 0.03454573263286213
Ext Val Set Accuracy: 0.5224691358024691
              precision    recall  f1-score   support

           0       0.51      0.29      0.37      2943
           1       0.53      0.74      0.62      3132

    accuracy                           0.52      6075
   macro avg       0.52      0.52      0.49      6075
weighted avg       0.52      0.52      0.50      6075

[[ 845 2098]
 [ 803 2329]]
5F-CV: 0.9201916500472332
FNA Set F1: 0.707011263554752
FNA 

### 3.3.3. GBM

In [73]:
## Build GBM Model
from sklearn.ensemble import GradientBoostingClassifier

def GBM_func(XTrain=X_train, XVal=X_val, XTest=X_FNA, yTrain=y_train, yVal=y_val, yTest=y_FNA):
    model_gbm = GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)
    model_gbm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_gbm.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_gbm.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_gbm.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_gbm.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_gbm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_gbm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_gbm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_gbm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_gbm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_gbm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_gbm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_gbm.feature_importances_, index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/XGB_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()

    # Visualize the boxplot of the important features between the two groups
    #fig = px.box(data_frame = df_ROI_for_ML_Opti, x = feature_imp[:10].index,color= 'type')
    #fig.show()
    
    return x, y

In [74]:
coeff_GBM, TOPfeatures_GBM = GBM_func()

Training Set F1: 0.9116094669891098
Training Set MCC: 0.8264573962800329
Training Set Accuracy: 0.9137093227792437
              precision    recall  f1-score   support

           0       0.91      0.93      0.92     47449
           1       0.92      0.90      0.91     43511

    accuracy                           0.91     90960
   macro avg       0.91      0.91      0.91     90960
weighted avg       0.91      0.91      0.91     90960

[[44079  3370]
 [ 4479 39032]]
Ext Val Set F1: 0.5429447705283154
Ext Val Set MCC: -0.10022450676481336
Ext Val Set Accuracy: 0.459917695473251
              precision    recall  f1-score   support

           0       0.41      0.26      0.32      2943
           1       0.48      0.65      0.55      3132

    accuracy                           0.46      6075
   macro avg       0.45      0.45      0.44      6075
weighted avg       0.45      0.46      0.44      6075

[[ 762 2181]
 [1100 2032]]
5F-CV: 0.7011533620526068
FNA Set F1: 0.5383865680391124
FNA

### 3.3.4. Stacking

In [75]:
## Build stacking Model

def Stacking_func(XTrain=X_train, XVal=X_val, XTest=X_FNA, XTest2=X_di, yTrain=y_train, yVal=y_val, yTest=y_FNA, yTest2=y_di):

    base_learners = []

    # Define models to use in stacking ensemble
    m1 = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})
    base_learners.append(m1)

    m2 = LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})
    base_learners.append(m2)

    m3 = GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)
    base_learners.append(m3)

    # Define the meta-classifier to use for stacking
    meta_learner = LogisticRegression(C=0.0011, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)

    model_sc = StackingRegressor([[m1, m2, m3], [meta_learner]])
    model_sc.fit(XTrain, yTrain)
    
    #=======================================
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_sc.predict(X_train)[-1]
    print("Training Set F1:", f1_score(y_train, model_sc.predict(X_train)[-1], sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_sc.predict(X_train)[-1], sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_sc.predict(XVal)[-1]
    print("Ext Val Set F1:", f1_score(yVal, model_sc.predict(XVal)[-1], sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_sc.predict(XVal)[-1], sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    #f1_5_train = cross_val_score(model_sc, XTrain, yTrain, cv = 5, scoring="f1")
    #print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_sc.predict(XTest)[-1]
    print("FNA Set F1:", f1_score(yTest, model_sc.predict(XTest)[-1], sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_sc.predict(XTest)[-1], sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_sc.predict(XTest)[-1]))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Test2 set
    predictions_di = model_sc.predict(XTest2)[-1]
    print("DirectIn Set F1:", f1_score(yTest2, model_sc.predict(XTest2)[-1], sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, model_sc.predict(XTest2)[-1], sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, model_sc.predict(XTest2)[-1]))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    

In [76]:
Stacking_func()

Training Set F1: 0.8679743714574002
Training Set MCC: 0.7325277019110886
Training Set Accuracy: 0.8655123131046614
              precision    recall  f1-score   support

           0       0.89      0.85      0.87     47449
           1       0.85      0.88      0.86     43511

    accuracy                           0.87     90960
   macro avg       0.87      0.87      0.87     90960
weighted avg       0.87      0.87      0.87     90960

[[40431  7018]
 [ 5215 38296]]
Ext Val Set F1: 0.7513333464863904
Ext Val Set MCC: 0.4810438528738775
Ext Val Set Accuracy: 0.7409053497942387
              precision    recall  f1-score   support

           0       0.75      0.69      0.72      2943
           1       0.73      0.79      0.76      3132

    accuracy                           0.74      6075
   macro avg       0.74      0.74      0.74      6075
weighted avg       0.74      0.74      0.74      6075

[[2035  908]
 [ 666 2466]]
FNA Set F1: 0.504465432926685
FNA Set MCC: -0.019348083916839

### 3.3.5. Voting

In [77]:
def Voting_func(XTrain=X_train, XVal=X_val, XTest=X_FNA, XTest2=X_di, yTrain=y_train, yVal=y_val, yTest=y_FNA, yTest2=y_di):

    models = get_models()
    
    ensemble = models["hard_voting"]
    ensemble.fit(XTrain, yTrain)
    #=======================================

    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = ensemble.predict(X_train)
    print("Training Set F1:", f1_score(y_train, ensemble.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, ensemble.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = ensemble.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, ensemble.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, ensemble.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()

    f1_5_train = cross_val_score(ensemble, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))

    # Test set
    predictions_FNA = ensemble.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, ensemble.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, ensemble.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, ensemble.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()

    # Test2 set
    predictions_di = ensemble.predict(XTest2)
    print("DirectIn Set F1:", f1_score(yTest2, ensemble.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, ensemble.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, ensemble.predict(XTest2)))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()

In [78]:
Voting_func()

Training Set F1: 0.9355460815781772
Training Set MCC: 0.8691645398438659
Training Set Accuracy: 0.933311345646438
              precision    recall  f1-score   support

           0       0.96      0.91      0.93     47449
           1       0.91      0.95      0.93     43511

    accuracy                           0.93     90960
   macro avg       0.93      0.93      0.93     90960
weighted avg       0.93      0.93      0.93     90960

[[43344  4105]
 [ 1961 41550]]
Ext Val Set F1: 0.6556326378591221
Ext Val Set MCC: 0.13720601034775723
Ext Val Set Accuracy: 0.5649382716049383
              precision    recall  f1-score   support

           0       0.62      0.27      0.37      2943
           1       0.55      0.85      0.67      3132

    accuracy                           0.56      6075
   macro avg       0.58      0.56      0.52      6075
weighted avg       0.58      0.56      0.52      6075

[[ 784 2159]
 [ 484 2648]]
5F-CV: 0.8703850133423877
FNA Set F1: 0.6041353313042247
FNA 

## 3.4 Machine Learning Modeling: Model 2 (Train 4:4, Val 6:6, Test 3:3)

### 3.4.1. Logistic Regression

In [79]:
## Build Logistic Regression Model
from sklearn.linear_model import LogisticRegression

def LogisticRegression_func(XTrain=X_val, XVal=X_train, XTest=X_FNA, yTrain=y_val, yVal=y_train, yTest=y_FNA):
    model_lr = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})  # max_iter=100
    model_lr.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_lr.predict(XVal)
    print("Training Set F1:", f1_score(yVal, model_lr.predict(XVal), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(yVal, model_lr.predict(XVal), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(yVal, X_train_predictions))
    print(classification_report(yVal, X_train_predictions))
    print(confusion_matrix(yVal, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yVal, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_lr.predict(XTrain)
    print("Ext Val Set F1:", f1_score(yTrain, model_lr.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yTrain, model_lr.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yTrain, predictions_val))
    print(classification_report(yTrain, predictions_val))
    print(confusion_matrix(yTrain, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yTrain, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_lr, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_lr.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_lr.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_lr.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_lr.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    return x, y

In [80]:
coeff_LogReg, TOPfeatures_LogReg = LogisticRegression_func()

Training Set F1: 0.0
Training Set MCC: 0.0
Training Set Accuracy: 0.5216468777484609
              precision    recall  f1-score   support

           0       0.52      1.00      0.69     47449
           1       0.00      0.00      0.00     43511

    accuracy                           0.52     90960
   macro avg       0.26      0.50      0.34     90960
weighted avg       0.27      0.52      0.36     90960

[[47449     0]
 [43511     0]]
Ext Val Set F1: 0.0
Ext Val Set MCC: 0.0
Ext Val Set Accuracy: 0.48444444444444446
              precision    recall  f1-score   support

           0       0.48      1.00      0.65      2943
           1       0.00      0.00      0.00      3132

    accuracy                           0.48      6075
   macro avg       0.24      0.50      0.33      6075
weighted avg       0.23      0.48      0.32      6075

[[2943    0]
 [3132    0]]
5F-CV: 0.0
FNA Set F1: 0.0
FNA Set MCC: 0.0
FNA Set Recall: 0.0
FNA Test Set Accuracy: 0.5021363907960451
              

### 3.4.2. SVM

In [81]:
## Build LinearSVC Model
from sklearn.svm import LinearSVC

def LinearSVC_func(XTrain=X_val, XVal=X_train, XTest=X_FNA, yTrain=y_val, yVal=y_train, yTest=y_FNA):
    model_svm = LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})
    model_svm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_svm.predict(XVal)
    print("Training Set F1:", f1_score(yVal, model_svm.predict(XVal), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(yVal, model_svm.predict(XVal), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(yVal, X_train_predictions))
    print(classification_report(yVal, X_train_predictions))
    print(confusion_matrix(yVal, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yVal, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_svm.predict(XTrain)
    print("Ext Val Set F1:", f1_score(yTrain, model_svm.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yTrain, model_svm.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yTrain, predictions_val))
    print(classification_report(yTrain, predictions_val))
    print(confusion_matrix(yTrain, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yTrain, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_svm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_svm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_svm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_svm.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_svm.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    return x, y

In [82]:
coeff_SVM, TOPfeatures_SVM = LinearSVC_func()

Training Set F1: 0.5866180873263486
Training Set MCC: 0.06628913385384579
Training Set Accuracy: 0.5262423043095866
              precision    recall  f1-score   support

           0       0.56      0.40      0.47     47449
           1       0.50      0.66      0.57     43511

    accuracy                           0.53     90960
   macro avg       0.53      0.53      0.52     90960
weighted avg       0.54      0.53      0.52     90960

[[18969 28480]
 [14613 28898]]
Ext Val Set F1: 0.951638273857085
Ext Val Set MCC: 0.9071510023599009
Ext Val Set Accuracy: 0.9520987654320988
              precision    recall  f1-score   support

           0       0.93      0.98      0.95      2943
           1       0.98      0.93      0.95      3132

    accuracy                           0.95      6075
   macro avg       0.95      0.95      0.95      6075
weighted avg       0.95      0.95      0.95      6075

[[2883   60]
 [ 231 2901]]
5F-CV: 0.8891078535936563
FNA Set F1: 0.4526933824725116
FNA 

### 3.4.3. GBM

In [83]:
## Build GBM Model
from sklearn.ensemble import GradientBoostingClassifier

def GBM_func(XTrain=X_val, XVal=X_train, XTest=X_FNA, yTrain=y_val, yVal=y_train, yTest=y_FNA):
    model_gbm = GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)
    model_gbm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_gbm.predict(XVal)
    print("Training Set F1:", f1_score(yVal, model_gbm.predict(XVal), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(yVal, model_gbm.predict(XVal), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(yVal, X_train_predictions))
    print(classification_report(yVal, X_train_predictions))
    print(confusion_matrix(yVal, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yVal, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_gbm.predict(XTrain)
    print("Ext Val Set F1:", f1_score(yTrain, model_gbm.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yTrain, model_gbm.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yTrain, predictions_val))
    print(classification_report(yTrain, predictions_val))
    print(confusion_matrix(yTrain, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yTrain, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_gbm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_gbm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_gbm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_gbm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_gbm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_gbm.feature_importances_, index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/XGB_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()

    # Visualize the boxplot of the important features between the two groups
    #fig = px.box(data_frame = df_ROI_for_ML_Opti, x = feature_imp[:10].index,color= 'type')
    #fig.show()
    
    return x, y

In [84]:
coeff_GBM, TOPfeatures_GBM = GBM_func()

Training Set F1: 0.7040687007871267
Training Set MCC: 0.3693306154115578
Training Set Accuracy: 0.6796174142480211
              precision    recall  f1-score   support

           0       0.73      0.61      0.67     47449
           1       0.64      0.75      0.69     43511

    accuracy                           0.68     90960
   macro avg       0.69      0.68      0.68     90960
weighted avg       0.69      0.68      0.68     90960

[[28976 18473]
 [10669 32842]]
Ext Val Set F1: 0.8843151945524823
Ext Val Set MCC: 0.7602080949794748
Ext Val Set Accuracy: 0.8791769547325103
              precision    recall  f1-score   support

           0       0.93      0.82      0.87      2943
           1       0.84      0.94      0.89      3132

    accuracy                           0.88      6075
   macro avg       0.89      0.88      0.88      6075
weighted avg       0.88      0.88      0.88      6075

[[2403  540]
 [ 194 2938]]
5F-CV: 0.6155215625599894
FNA Set F1: 0.6301864421122367
FNA 

### 3.4.4. Stacking

In [85]:
## Build stacking Model

def Stacking_func(XTrain=X_val, XVal=X_train, XTest=X_FNA, XTest2=X_di, yTrain=y_val, yVal=y_train, yTest=y_FNA, yTest2=y_di):

    base_learners = []

    # Define models to use in stacking ensemble
    m1 = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})
    base_learners.append(m1)

    m2 = LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})
    base_learners.append(m2)

    m3 = GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)
    base_learners.append(m3)

    # Define the meta-classifier to use for stacking
    meta_learner = LogisticRegression(C=0.0011, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)

    model_sc = StackingRegressor([[m1, m2, m3], [meta_learner]])
    model_sc.fit(XTrain, yTrain)
    
    #=======================================
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_sc.predict(XVal)[-1]
    print("Training Set F1:", f1_score(yVal, model_sc.predict(XVal)[-1], sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(yVal, model_sc.predict(XVal)[-1], sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(yVal, X_train_predictions))
    print(classification_report(yVal, X_train_predictions))
    print(confusion_matrix(yVal, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yVal, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_sc.predict(XTrain)[-1]
    print("Ext Val Set F1:", f1_score(yTrain, model_sc.predict(XTrain)[-1], sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yTrain, model_sc.predict(XTrain)[-1], sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yTrain, predictions_val))
    print(classification_report(yTrain, predictions_val))
    print(confusion_matrix(yTrain, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yTrain, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    #f1_5_train = cross_val_score(model_sc, XTrain, yTrain, cv = 5, scoring="f1")
    #print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_sc.predict(XTest)[-1]
    print("FNA Set F1:", f1_score(yTest, model_sc.predict(XTest)[-1], sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_sc.predict(XTest)[-1], sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_sc.predict(XTest)[-1]))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Test2 set
    predictions_di = model_sc.predict(XTest2)[-1]
    print("DirectIn Set F1:", f1_score(yTest2, model_sc.predict(XTest2)[-1], sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, model_sc.predict(XTest2)[-1], sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, model_sc.predict(XTest2)[-1]))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    

In [86]:
Stacking_func()

Training Set F1: 0.0
Training Set MCC: 0.0
Training Set Accuracy: 0.5216468777484609
              precision    recall  f1-score   support

           0       0.52      1.00      0.69     47449
           1       0.00      0.00      0.00     43511

    accuracy                           0.52     90960
   macro avg       0.26      0.50      0.34     90960
weighted avg       0.27      0.52      0.36     90960

[[47449     0]
 [43511     0]]
Ext Val Set F1: 0.0
Ext Val Set MCC: 0.0
Ext Val Set Accuracy: 0.48444444444444446
              precision    recall  f1-score   support

           0       0.48      1.00      0.65      2943
           1       0.00      0.00      0.00      3132

    accuracy                           0.48      6075
   macro avg       0.24      0.50      0.33      6075
weighted avg       0.23      0.48      0.32      6075

[[2943    0]
 [3132    0]]
FNA Set F1: 0.0
FNA Set MCC: 0.0
FNA Set Recall: 0.0
FNA Test Set Accuracy: 0.5021363907960451
              precision  

### 3.4.5. Voting

In [87]:
def Voting_func(XTrain=X_val, XVal=X_train, XTest=X_FNA, XTest2=X_di, yTrain=y_val, yVal=y_train, yTest=y_FNA, yTest2=y_di):

    models = get_models()
    
    ensemble = models["hard_voting"]
    ensemble.fit(XTrain, yTrain)
    #=======================================

    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = ensemble.predict(XVal)
    print("Training Set F1:", f1_score(yVal, ensemble.predict(XVal), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(yVal, ensemble.predict(XVal), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(yVal, X_train_predictions))
    print(classification_report(yVal, X_train_predictions))
    print(confusion_matrix(yVal, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yVal, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = ensemble.predict(XTrain)
    print("Ext Val Set F1:", f1_score(yTrain, ensemble.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yTrain, ensemble.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yTrain, predictions_val))
    print(classification_report(yTrain, predictions_val))
    print(confusion_matrix(yTrain, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yTrain, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()

    f1_5_train = cross_val_score(ensemble, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))

    # Test set
    predictions_FNA = ensemble.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, ensemble.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, ensemble.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, ensemble.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()

    # Test2 set
    predictions_di = ensemble.predict(XTest2)
    print("DirectIn Set F1:", f1_score(yTest2, ensemble.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, ensemble.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, ensemble.predict(XTest2)))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()

In [88]:
Voting_func()

Training Set F1: 0.5785390636495318
Training Set MCC: 0.2721761428655857
Training Set Accuracy: 0.6371481970096746
              precision    recall  f1-score   support

           0       0.63      0.76      0.69     47449
           1       0.66      0.51      0.57     43511

    accuracy                           0.64     90960
   macro avg       0.64      0.63      0.63     90960
weighted avg       0.64      0.64      0.63     90960

[[35957 11492]
 [21513 21998]]
Ext Val Set F1: 0.9310661601937893
Ext Val Set MCC: 0.8752227341100146
Ext Val Set Accuracy: 0.9331687242798354
              precision    recall  f1-score   support

           0       0.89      0.99      0.93      2943
           1       0.99      0.88      0.93      3132

    accuracy                           0.93      6075
   macro avg       0.94      0.93      0.93      6075
weighted avg       0.94      0.93      0.93      6075

[[2915   28]
 [ 378 2754]]
5F-CV: 0.6234347986181539
FNA Set F1: 0.4585361994802933
FNA 

# 4. Model Evaluation

## 4.1 Prediction

In [89]:
## deploy models for training set ##
model_voting = jl.load(r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\dbMSIn_5ppm\model3to1Voting_dbMSIn5ppm.joblib")
#
## deploy model ##
predicted0n1_train = model_voting.predict(X_train)
#predicted0n1_trainProba = model_voting.predict_proba(X_train)         
trainDEFSDf["predicted0n1"] = predicted0n1_train
T_ = []
F_ = []
for i in range(len(trainDEFSDf)):
    if trainDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif trainDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
trainDEFSDf["T"] = T_
trainDEFSDf["F"] = F_
#
## save ##, ouputing trainSet df
savePath = r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\dbMSIn_5ppm\df_train_dbMSIn5ppm_norm_0n1.csv"
trainDEFSDf.to_csv(savePath)

In [90]:
## deploy models for Ext Val set ##
model_voting = jl.load(r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\dbMSIn_5ppm\model3to1Voting_dbMSIn5ppm.joblib")
#
## deploy model ##
predicted0n1_ext = model_voting.predict(X_val)
extDEFSDf["predicted0n1"] = predicted0n1_ext
T_ = []
F_ = []
for i in range(len(extDEFSDf)):
    if extDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif extDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
extDEFSDf["T"] = T_
extDEFSDf["F"] = F_
#
## save ##, ouputing extSet df
savePath = r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\dbMSIn_5ppm\df_ext_dbMSIn5ppm_norm_0n1.csv"
extDEFSDf.to_csv(savePath)

In [91]:
## deploy models for FNA set ##
model_voting = jl.load(r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\dbMSIn_5ppm\model3to1Voting_dbMSIn5ppm.joblib")
#
## deploy model ##
predicted0n1_FNA = model_voting.predict(X_FNA)
fnaDEFSDf["predicted0n1"] = predicted0n1_FNA
T_ = []
F_ = []
for i in range(len(fnaDEFSDf)):
    if fnaDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif fnaDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
fnaDEFSDf["T"] = T_
fnaDEFSDf["F"] = F_
#
## save ##, ouputing extSet df
savePath = r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\dbMSIn_5ppm\df_FNA_dbMSIn5ppm_norm_0n1.csv"
fnaDEFSDf.to_csv(savePath)

In [92]:
## deploy models for DirectInfusion set ##
model_voting = jl.load(r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\dbMSIn_5ppm\model3to1Voting_dbMSIn5ppm.joblib")
#
## deploy model ##
predicted0n1_Di = model_voting.predict(X_di)
diDEFSDf["predicted0n1"] = predicted0n1_Di
T_ = []
F_ = []
for i in range(len(diDEFSDf)):
    if diDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif diDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
diDEFSDf["T"] = T_
diDEFSDf["F"] = F_
#
## save ##, ouputing extSet df
savePath = r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\dbMSIn_5ppm\df_Di_dbMSIn5ppm_norm_0n1.csv"
diDEFSDf.to_csv(savePath)